In [2]:
from utils import get_answer, followup_prompt

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [29]:
function_name="lib_echo"
IS_C_CODE= True
lib_files = ['/home/sagemaker-user/afl-training/harness/library.c','/home/sagemaker-user/afl-training/harness/library.h']
HARNESS = 'harness.c'
HARNESS_BIN = 'harness'


In [4]:
lib_for_prompt = ''

code_md = '```c' if IS_C_CODE else '```c++'
for l in lib_files:
    with open(l) as file:
        content = f"\n{l}\n{code_md}\n{file.read()}\n```\n"
    lib_for_prompt = lib_for_prompt + content
lib_for_prompt=f'''
## Library:
{lib_for_prompt}
## End of library
'''

In [48]:

role = f"You are a developer. you need to write a libFuzzer target for the {function_name} function of your c library."

fill_c = '''
<FILL>

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
'''
fill_cpp = '''
<FILL>

extern "C" int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
'''
template= '''
#include <stdint.h>
#include <stdlib.h>
'''+'''
// Step 2 include custom and standard headers
'''+ fill_c if IS_C_CODE else fill_c_cp+f'''
// Step 3 call to the {function_name} function
<FILL>;
''' +'''
  return 0;
}
'''
graph='}'
initial_prompt = f"""
{role}

Work step by step:
1. Identify and analyze {function_name} in the library source code: which headers and parameters does it need to be properly executed?
2. Fill the template with necessary headers
3. fill the template LLVMFuzzerTestOneInput function with a call to {function_name}, such that {function_name} can be properly fuzz
Answer should be code only, no explanation.

{lib_for_prompt}

## template:
{code_md}
{template}
```
## End of template:
 """
initial_prompt = '[INST]' + initial_prompt + '[/INST]'

In [49]:
# payload = {
#     "inputs": "import socket\n\ndef ping_exponential_backoff(host: str):",
#     "parameters": {
#         "max_new_tokens": 256,
#         "top_p": 0.9,
#         "temperature": 0.2
#     }
# }
# response = predictor.predict(payload)
# print(response)

In [50]:
# print(response[0]['generated_text'])

In [51]:
# print(r[0]['generated_text'])

In [52]:
import re

def extract_summary_stats(text):
    stats_start = text.find('Summary stats')
    if stats_start == -1:
        return {}  # Return an empty dictionary if the section is not found
    
    # Isolate the section by finding the section title and the next occurrence of '==', or the end of the file
    stats_section = text[stats_start:]
    # stats_end = stats_section.find('\n\n', stats_section.find('\n'))  # Find end by looking for next double newline after first newline
    # if stats_end == -1:
    #     stats_end = len(stats_section)
    # else:
    #     stats_section = stats_section[:stats_end]
    
    # Extract each statistic line
    lines = stats_section.split('\n')[2:]  # Skip the title and any empty line directly after it
    
    # Define a dictionary to hold the stats
    stats_dict = {}
    
    # Regex to capture key and value
    stat_regex = r'^\s*(.*?):\s*(.*)$'
    
    # Process each line
    for line in lines:
        match = re.match(stat_regex, line)
        if match:
            key = match.group(1).strip()
            value = match.group(2).strip()
    
            # Normalize the key to a consistent format
            key = ' '.join(key.lower().split())
    
            # Convert numbers if possible
            if value.replace('.', '', 1).isdigit():  # Check if the value is a number
                if '.' in value:
                    value = float(value)
                else:
                    value = int(value)
    
            # Store in dictionary
            stats_dict[key] = value


    return stats_dict

In [53]:

def extract_and_save_harness(llm_res):
    lines = llm_res[0]['generated_text'].split('\n')
    start_idx = next(i for i, line in enumerate(lines) if '```' in line) + 1
    end_idx = next(i for i, line in enumerate(lines) if i > start_idx and '```' in line)
    c_code = '\n'.join(lines[start_idx:end_idx])
    with open(HARNESS, 'w') as file:
        file.write(c_code)

In [68]:
import subprocess
compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', HARNESS, 'library.c', '-o', HARNESS_BIN]

def compile(compile_command):
    print('checking that harness is compilable')
    print(f"{compile_command=}")
    result = subprocess.run(compile_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        print("Compilation successful.")
        return True, ""
    else:
        print("Compilation failed.")
        print("Errors:", result.stderr)
        # with open('error.log', 'w') as file:
        #     file.write(result.stderr)
        return False, result.stderr

def run_seed():
    print('checking that does not crash on seed')
    result = subprocess.run([f'./{HARNESS_BIN}', 'seed/seed'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        return True, ""
    else:
        print("Errors:", result.stderr)
        return False, result.stderr

def has_coverage():
    seconds_to_wait = 15
    print(f'checking that there is some coverage (Running the fuzzer for {seconds_to_wait}) seconds')
    # print(datetime.now())
    # r = subprocess.Popen(['afl-fuzz', '-i', 'seed', '-o', 'out', '--', f'./{HARNESS_BIN}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # time.sleep(seconds_to_wait)
    # r.terminate()
    # try:
    #     r.wait(timeout=2)  # Give some time to clean up resources
    # except subprocess.TimeoutExpired:
    #     r.kill()  # Force kill if it's still not terminating
    # print(datetime.now())
    !timeout 5s afl-fuzz -i seed -o out -- ./harness > /dev/null
    result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stats = extract_summary_stats(result.stdout)
    print(f"{stats['coverage reached'][:-1]}")
    if float(stats['coverage reached'][:-1]) == 0:
        print('Trying to fuzz for 60 seconds')
        !timeout 60s afl-fuzz -i seed -o out -- ./harness > /dev/null
        result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stats = extract_summary_stats(result.stdout)
        print(f"{stats['coverage reached'][:-1]}")  
    return not float(stats['coverage reached'][:-1]) == 0

In [67]:
tentatives = 10
first_answer = get_answer(initial_prompt)
extract_and_save_harness(first_answer)
prev_prompt = initial_prompt
prev_res = first_answer[0]['generated_text']

# compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', HARNESS, 'library.c', '-o', 'file']

while tentatives:
    print(f"{tentatives=}")
    tentatives = tentatives -1
    # checking that harness is compilable
    compile_res = compile(compile_command)
    if compile_res[0]:
    # checking that does not crash on seed
        run_res = run_seed()
        if run_res[0]:
            # checking that there is some coverage
            if has_coverage():
                break
            else:
                prev_prompt = followup_prompt(prev_prompt, prev_res, "When running afl-fuzz over the generated code, the Coverage reached value is 0%. It probably means that the input from the fuzzer is not correclty passed to the fuzzed function. Can you regenerate the code to fiX this issue?\n")
                prev_res = get_answer(prev_prompt)
                extract_and_save_harness(prev_res)
        else:
            print(f"{run_res[1]=}")
            prev_prompt = followup_prompt(prev_prompt, prev_res, "The generated code when run with input AAAA is crashing with this error\n" + run_res[1] + '\ncan you regenerate the previous code to fix the issue?')
            prev_res = get_answer(prev_prompt)
            extract_and_save_harness(prev_res)
    else:
        print(f"{compile_res[1]=}")
        prev_prompt = followup_prompt(prev_prompt, prev_res, "can you regenerate the previous code to fix the following compilation error:\n" + compile_res[1])
        prev_res = get_answer(prev_prompt)
        extract_and_save_harness(prev_res)
        # break

========= PAYLOAD =========
payload = {
    "inputs": "[INST]\nYou are a developer. you need to write a libFuzzer target for the lib_echo function of your c library.\n\nWork step by step:\n1. Identify and analyze lib_echo in the library source code: which headers and parameters does it need to be properly executed?\n2. Fill the template with necessary headers\n3. fill the template LLVMFuzzerTestOneInput function with a call to lib_echo, such that lib_echo can be properly fuzz\nAnswer should be code only, no explanation.\n\n\n## Library:\n\n/home/sagemaker-user/afl-training/harness/library.c\n```c\n#include <stdlib.h>\n#include <stdio.h>\n#include <string.h>\n#include <assert.h>\n\n#include \"library.h\"\n\nvoid lib_echo(char *data, ssize_t len){\n\tif(strlen(data) == 0) {\n\t\treturn;\n\t}\n\tchar *buf = calloc(1, len);\n\tstrncpy(buf, data, len);\n\tprintf(\"%s\",buf);\n\tfree(buf);\n\n\t// A crash so we can tell the harness is working for lib_echo\n\tif(data[0] == 'p') {\n\t\tif(data

In [56]:
# r = get_answer(fix_compilation_prompt)

0
/usr/local/bin/afl-whatsup status check tool for afl-fuzz by Michal Zalewski

Individual fuzzers

>>> ./file instance: default (0 days, 0 hrs) fuzzer PID: 26820 <<<

  last_find       : 19830 days, 4 hours
  last_crash      : 19830 days, 4 hours
  last_hang       : none seen yet
  cycles_wo_finds : 461
  coverage        : 0.02%
  cpu usage 38.6%, memory usage 0.7%
  cycles 482, lifetime speed 32958 execs/sec, items 4/5 (80%)
  pending 0/0, coverage 0.02%, crashes saved 1 (!)

Summary stats

        Fuzzers alive : 1
       Total run time : 1 minutes, 0 seconds
          Total execs : 1 millions, 977 thousands
     Cumulative speed : 32958 execs/sec
  Total average speed : 32958 execs/sec
Current average speed : 32345 execs/sec
        Pending items : 0 faves, 0 total
     Coverage reached : 0.02%
        Crashes saved : 1
          Hangs saved : 0
 Cycles without finds : 461
   Time without finds : 19830 days, 4 hours




0.02

In [92]:
result.stdout

'/usr/local/bin/afl-whatsup status check tool for afl-fuzz by Michal Zalewski\n\nIndividual fuzzers\n==================\n\n>>> ./file instance: default (0 days, 0 hrs) fuzzer PID: 26820 <<<\n\n  Instance is dead or running remotely, skipping.\n\nSummary stats\n=============\n\n        Fuzzers alive : 0\n       Dead or remote : 1 (excluded from stats)\n       Total run time : 0 seconds\n          Total execs : 0 thousands\n     Cumulative speed : 0 execs/sec\n        Pending items : 0 faves, 0 total\n     Coverage reached : 0.02%\n        Crashes saved : 0\n          Hangs saved : 0\n Cycles without finds : 685\n   Time without finds : 0\n\n'